In [55]:
#### pollution data 정리
import pandas as pd
import numpy as np

##PM2.5
df_pm25 = pd.read_csv('PM2.5.csv')

# 열 이름 변경
df_pm25.rename(columns={'구분(2)': 'City', '구분(1)': 'District'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_pm25 = df_pm25[~df_pm25['City'].isin(['소계', '도평균'])]
df_pm25.reset_index(drop=True, inplace=True)

# 2010.01부터 2014.12까지의 월별 열 생성 ( 없는 날짜 만들어서 nan 값으로 채우기)
date_range_2010_2014 = pd.date_range(start='2010-01-01', end='2014-12-01', freq='MS').strftime('%Y.%m').tolist()

district_city = df_pm25[['District', 'City']]
date_range_2015_2022 = pd.date_range(start='2015-01-01', end='2022-11-01', freq='MS').strftime('%Y.%m').tolist()
existing_columns_2015_2022 = [col for col in df_pm25.columns if col in date_range_2015_2022]

# 2010.01부터 2014.12까지의 월별 열을 새로운 데이터프레임으로 생성
df_date_2010_2014 = pd.DataFrame(columns=date_range_2010_2014)

# 없는 날짜는 NaN 값으로
df_date_2010_2014 = df_date_2010_2014.reindex(columns=date_range_2010_2014).fillna(np.nan)

# District, City, 날짜순으로 새로 정렬해서 dataset 완성
df_pm25 = pd.concat([district_city, df_date_2010_2014, df_pm25[existing_columns_2015_2022]], axis=1)


# 변환할 날짜 열 리스트( district와 city를 제외한 모든 월 선택 )
date_columns = [col for col in df_pm25.columns if col not in ['District', 'City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / pm25에 값 저장
df_changed = pd.melt(df_pm25, id_vars=['District', 'City'], value_vars=date_columns, var_name='date', value_name='pm25')

# 날짜가 2015년부터 2022년까지 .1로 끝나는 경우 뒤에 0을 붙여 .10으로 변환
df_changed['date'] = df_changed['date'].apply(lambda x: x + '0' if x.endswith('.1') else x)

print(df_changed)

      District     City     date pm25
0        서울특별시    서울특별시  2010.01  NaN
1        부산광역시    부산광역시  2010.01  NaN
2        대구광역시    대구광역시  2010.01  NaN
3        인천광역시    인천광역시  2010.01  NaN
4        광주광역시    광주광역시  2010.01  NaN
...        ...      ...      ...  ...
23809     경상남도       의령  2022.11   22
23810     경상남도       창녕  2022.11   21
23811     경상남도       합천  2022.11   21
23812  제주특별자치도  제주특별자치도  2022.11   20
23813  제주특별자치도      서귀포  2022.11   14

[23814 rows x 4 columns]


In [56]:
##PM10
df_pm10 = pd.read_csv('PM10.csv')

# 열 이름 변경
df_pm10.rename(columns={'구분': 'City'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_pm10 = df_pm10[~df_pm10['City'].isin(['총계', '도평균'])]
df_pm10.reset_index(drop=True, inplace=True)
df_pm10 = df_pm10.iloc[:-1]  # 마지막 행 삭제

# '월' 제거
df_pm10.columns = [col.replace(' 월', '') for col in df_pm10.columns]
df_pm10.to_csv('test.csv')

# "District" 열 추가
df_pm10.insert(0, "District", "")

# 모든 열의 데이터 타입을 "object"로 변경
df_pm10 = df_pm10.astype(object)

# District 열 채우기
df_pm10['District'] = df_pm10.apply(lambda row: df_pm25.loc[df_pm25['City'] == row['City'], 'District'].values[0] if row['City'] in df_pm25['City'].values else np.nan, axis=1)

# 변환할 날짜 열 리스트( city, district 를 제외한 모든 월 선택 )
date_columns = [col for col in df_pm10.columns if col not in ['District','City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / pm10에 값 저장
df_pm10 = pd.melt(df_pm10, id_vars=['District','City'], value_vars=date_columns, var_name='date', value_name='pm10')

# 기존 데이터프레임(df_pm25)과 새로운 데이터프레임(df_pm10)을 합치기
df_merged = pd.merge(df_changed, df_pm10, on=['District','City', 'date'], how='outer')

# 정렬하고 pm25에는 없는 city를 지웠음
df_merged = df_merged.sort_values(['District', 'City'], ignore_index=True)
df_merged = df_merged[~df_merged['City'].isin(['마산', '진해', '청원'])]

print(df_merged)

      District City     date pm25  pm10
0          강원도   강릉  2010.01  NaN  48.0
1          강원도   강릉  2010.02  NaN    51
2          강원도   강릉  2010.03  NaN  62.0
3          강원도   강릉  2010.04  NaN    51
4          강원도   강릉  2010.05  NaN    51
...        ...  ...      ...  ...   ...
25252     충청북도   충주  2018.10  NaN    48
25253     충청북도   충주  2019.10  NaN    34
25254     충청북도   충주  2020.10  NaN    35
25255     충청북도   충주  2021.10  NaN    27
25256     충청북도   충주  2022.10  NaN    26

[25257 rows x 5 columns]


In [57]:
##SO2
df_so2 = pd.read_csv('SO2.csv')

# 열 이름 변경
df_so2.rename(columns={'구분': 'City'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_so2 = df_so2[~df_so2['City'].isin(['총계', '도평균'])]
df_so2.reset_index(drop=True, inplace=True)
df_so2 = df_so2.iloc[:-1]  # 마지막 행 삭제

# '월' 제거
df_so2.columns = [col.replace(' 월', '') for col in df_so2.columns]

# "District" 열 추가
df_so2.insert(0, "District", "")

# 모든 열의 데이터 타입을 "object"로 변경
df_so2 = df_so2.astype(object)

# District 열 채우기
df_so2['District'] = df_so2.apply(lambda row: df_merged.loc[df_merged['City'] == row['City'], 'District'].values[0] if row['City'] in df_merged['City'].values else np.nan, axis=1)

# 변환할 날짜 열 리스트 (city, district를 제외한 모든 월 선택)
date_columns = [col for col in df_so2.columns if col not in ['District', 'City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / so2에 값 저장
df_so2 = pd.melt(df_so2, id_vars=['District', 'City'], value_vars=date_columns, var_name='date', value_name='so2')

# 기존 데이터프레임과 새로운 데이터프레임(df_so2)을 합치기
df_merged = pd.merge(df_merged, df_so2, on=['District', 'City', 'date'], how='outer')

# 정렬하고 pm25에는 없는 city를 지웠음
df_merged = df_merged.sort_values(['District', 'City'], ignore_index=True)
df_merged = df_merged[~df_merged['City'].isin(['마산', '진해', '청원'])]

print(df_merged)

      District City     date pm25  pm10    so2
0          강원도   강릉  2010.01  NaN  48.0  0.007
1          강원도   강릉  2010.02  NaN    51  0.006
2          강원도   강릉  2010.03  NaN  62.0  0.006
3          강원도   강릉  2010.04  NaN    51  0.006
4          강원도   강릉  2010.05  NaN    51  0.004
...        ...  ...      ...  ...   ...    ...
25562     충청북도   충주  2018.10  NaN    48  0.003
25563     충청북도   충주  2019.10  NaN    34  0.004
25564     충청북도   충주  2020.10  NaN    35  0.003
25565     충청북도   충주  2021.10  NaN    27  0.002
25566     충청북도   충주  2022.10  NaN    26  0.002

[25567 rows x 6 columns]


In [58]:
##O3
df_o3 = pd.read_csv('O3.csv')

# 열 이름 변경
df_o3.rename(columns={'구분': 'City'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_o3 = df_o3[~df_o3['City'].isin(['총계', '도평균'])]
df_o3.reset_index(drop=True, inplace=True)
df_o3 = df_o3.iloc[:-1]  # 마지막 행 삭제

# '월' 제거
df_o3.columns = [col.replace(' 월', '') for col in df_o3.columns]

# "District" 열 추가
df_o3.insert(0, "District", "")

# 모든 열의 데이터 타입을 "object"로 변경
df_o3 = df_o3.astype(object)

# District 열 채우기
df_o3['District'] = df_o3.apply(lambda row: df_merged.loc[df_merged['City'] == row['City'], 'District'].values[0] if row['City'] in df_merged['City'].values else np.nan, axis=1)

# 변환할 날짜 열 리스트 (city, district를 제외한 모든 월 선택)
date_columns = [col for col in df_o3.columns if col not in ['District', 'City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / o3에 값 저장
df_o3 = pd.melt(df_o3, id_vars=['District', 'City'], value_vars=date_columns, var_name='date', value_name='o3')

# 기존 데이터프레임과 새로운 데이터프레임(df_o3)을 합치기
df_merged = pd.merge(df_merged, df_o3, on=['District', 'City', 'date'], how='outer')

# 정렬하고 pm25에는 없는 city를 지웠음
df_merged = df_merged.sort_values(['District', 'City'], ignore_index=True)
df_merged = df_merged[~df_merged['City'].isin(['마산', '진해', '청원'])]

print(df_merged)

      District City     date pm25  pm10    so2     o3
0          강원도   강릉  2010.01  NaN  48.0  0.007  0.024
1          강원도   강릉  2010.02  NaN    51  0.006  0.022
2          강원도   강릉  2010.03  NaN  62.0  0.006  0.028
3          강원도   강릉  2010.04  NaN    51  0.006  0.036
4          강원도   강릉  2010.05  NaN    51  0.004  0.044
...        ...  ...      ...  ...   ...    ...    ...
26182     충청북도   충주  2018.10  NaN    48  0.003  0.019
26183     충청북도   충주  2019.10  NaN    34  0.004  0.021
26184     충청북도   충주  2020.10  NaN    35  0.003  0.023
26185     충청북도   충주  2021.10  NaN    27  0.002  0.022
26186     충청북도   충주  2022.10  NaN    26  0.002  0.021

[26187 rows x 7 columns]


In [59]:
##NO2
df_no2 = pd.read_csv('NO2.csv')

# 열 이름 변경
df_no2.rename(columns={'구분': 'City'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_no2 = df_no2[~df_no2['City'].isin(['총계', '도평균'])]
df_no2.reset_index(drop=True, inplace=True)
df_no2 = df_no2.iloc[:-1]  # 마지막 행 삭제

# '월' 제거
df_no2.columns = [col.replace(' 월', '') for col in df_no2.columns]

# "District" 열 추가
df_no2.insert(0, "District", "")

# 모든 열의 데이터 타입을 "object"로 변경
df_no2 = df_no2.astype(object)

# District 열 채우기
df_no2['District'] = df_no2.apply(lambda row: df_merged.loc[df_merged['City'] == row['City'], 'District'].values[0] if row['City'] in df_merged['City'].values else np.nan, axis=1)

# 변환할 날짜 열 리스트 (city, district를 제외한 모든 월 선택)
date_columns = [col for col in df_no2.columns if col not in ['District', 'City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / no2에 값 저장
df_no2 = pd.melt(df_no2, id_vars=['District', 'City'], value_vars=date_columns, var_name='date', value_name='no2')

# 기존 데이터프레임과 새로운 데이터프레임(df_no2)을 합치기
df_merged = pd.merge(df_merged, df_no2, on=['District', 'City', 'date'], how='outer')

# 정렬하고 pm25에는 없는 city를 지웠음
df_merged = df_merged.sort_values(['District', 'City'], ignore_index=True)
df_merged = df_merged[~df_merged['City'].isin(['마산', '진해', '청원'])]

print(df_merged)


      District City     date pm25  pm10    so2     o3     no2
0          강원도   강릉  2010.01  NaN  48.0  0.007  0.024    0.02
1          강원도   강릉  2010.02  NaN    51  0.006  0.022   0.022
2          강원도   강릉  2010.03  NaN  62.0  0.006  0.028    0.02
3          강원도   강릉  2010.04  NaN    51  0.006  0.036    0.02
4          강원도   강릉  2010.05  NaN    51  0.004  0.044   0.017
...        ...  ...      ...  ...   ...    ...    ...     ...
27422     충청북도   충주  2018.10  NaN    48  0.003  0.019   0.023
27423     충청북도   충주  2019.10  NaN    34  0.004  0.021   0.019
27424     충청북도   충주  2020.10  NaN    35  0.003  0.023   0.015
27425     충청북도   충주  2021.10  NaN    27  0.002  0.022  0.014*
27426     충청북도   충주  2022.10  NaN    26  0.002  0.021   0.014

[27427 rows x 8 columns]


In [60]:
##CO
df_co = pd.read_csv('CO.csv')

# 열 이름 변경
df_co.rename(columns={'구분': 'City'}, inplace=True)

# 총계, 소계, 도평균 삭제
df_co = df_co[~df_co['City'].isin(['총계', '도평균'])]
df_co.reset_index(drop=True, inplace=True)
df_co = df_co.iloc[:-1]  # 마지막 행 삭제

# '월' 제거
df_co.columns = [col.replace(' 월', '') for col in df_co.columns]

# "District" 열 추가
df_co.insert(0, "District", "")

# 모든 열의 데이터 타입을 "object"로 변경
df_co = df_co.astype(object)

# District 열 채우기
df_co['District'] = df_co.apply(lambda row: df_merged.loc[df_merged['City'] == row['City'], 'District'].values[0] if row['City'] in df_merged['City'].values else np.nan, axis=1)

# 변환할 날짜 열 리스트 (city, district를 제외한 모든 월 선택)
date_columns = [col for col in df_co.columns if col not in ['District', 'City']]

# 날짜 열을 행별로 변환하여 "date" 열 생성 / co에 값 저장
df_co = pd.melt(df_co, id_vars=['District', 'City'], value_vars=date_columns, var_name='date', value_name='co')

# 기존 데이터프레임과 새로운 데이터프레임(df_co)을 합치기
df_merged = pd.merge(df_merged, df_co, on=['District', 'City', 'date'], how='outer')

# 정렬하고 pm25에는 없는 city를 지웠음
df_merged = df_merged.sort_values(['District', 'City'], ignore_index=True)
df_merged = df_merged[~df_merged['City'].isin(['마산', '진해', '청원'])]

print(df_merged)

df_merged.to_csv('pollution_final.csv')

      District City     date pm25  pm10    so2     o3     no2   co
0          강원도   강릉  2010.01  NaN  48.0  0.007  0.024    0.02  0.6
1          강원도   강릉  2010.02  NaN    51  0.006  0.022   0.022  0.6
2          강원도   강릉  2010.03  NaN  62.0  0.006  0.028    0.02  0.5
3          강원도   강릉  2010.04  NaN    51  0.006  0.036    0.02  0.5
4          강원도   강릉  2010.05  NaN    51  0.004  0.044   0.017  0.4
...        ...  ...      ...  ...   ...    ...    ...     ...  ...
29902     충청북도   충주  2018.10  NaN    48  0.003  0.019   0.023  0.3
29903     충청북도   충주  2019.10  NaN    34  0.004  0.021   0.019  0.5
29904     충청북도   충주  2020.10  NaN    35  0.003  0.023   0.015  0.4
29905     충청북도   충주  2021.10  NaN    27  0.002  0.022  0.014*  0.4
29906     충청북도   충주  2022.10  NaN    26  0.002  0.021   0.014  0.4

[29907 rows x 9 columns]
